In [7]:
# Use centrailzed training to compare with federated learning
epochs = 1
n_train_items = 12800

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np

# import torchcsprng as csprng


In [3]:
torch.cuda.is_available()

False

In [11]:
class Arguments():
    def __init__(self):
        self.batch_size = 250
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.25
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# random_number_generator = csprng.create_random_device_generator(
#                 "/dev/urandom")

In [5]:
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

# train_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(train_loader)
#         if i < n_train_items / args.batch_size
#     ]


# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

    def name(self):
        return "SampleConvNet"

In [8]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.5)
        
        # adding noise to the gradient
        for param in model.parameters():
            noise = torch.normal(0,1.3*1.5,param.size(),device=device)*args.lr/args.batch_size
            param.grad.add_(noise)
        optimizer.step()

#           param.add_(torch.from_numpy(np.random.normal(0,1.3*1.5,param.size())*args.lr).to(device))
        
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader) * args.batch_size,
                100. * batch_idx / len(train_dataloader), loss.item()))

In [9]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.test_batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.test_batch_size),
        100. * correct / (len(test_loader)*args.test_batch_size)))


In [101]:
def compare_model(m1, m2):
    m1s = m1.state_dict()
    m2s = m2.state_dict()
    for key in m1s:
        print("std:{:.4f}".format(np.std((m1s[key]-m2s[key]).numpy(), ddof = 1)))
        print("mean:{:.2f}".format(np.mean((m1s[key]-m2s[key]).numpy())))


In [102]:
import opacus
from opacus import PrivacyEngine

In [162]:
import copy
model_mine = Net()
model_opacus = copy.deepcopy(model_mine)
optimizer =  optim.SGD(model_opacus.parameters(), lr=args.lr)

model_nondp = copy.deepcopy(model_mine)
opt_nondp =  optim.SGD(model_nondp.parameters(), lr=args.lr)


privacy_engine = PrivacyEngine(
                model_opacus,
                batch_size=args.batch_size,
                sample_size=len(train_dataloader.dataset),
                alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
                noise_multiplier=2,
                max_grad_norm=1.5,)
privacy_engine.attach(optimizer)


data = train_dataloader.__iter__().next()[0]
target = train_dataloader.__iter__().next()[1]
output = model_opacus(data)
loss = F.cross_entropy(output, target)
loss.backward()
torch.nn.utils.clip_grad_norm_(model_opacus.parameters(),1.5)
optimizer.step()

data = train_dataloader.__iter__().next()[0]
target = train_dataloader.__iter__().next()[1]
output = model_nondp(data)
loss = F.cross_entropy(output, target)
loss.backward()
torch.nn.utils.clip_grad_norm_(model_nondp.parameters(),1.5)
opt_nondp.step()



compare_model(model_mine, model_opacus)
print('#####')
compare_model(model_opacus, model_nondp)

/opt/conda/envs/DP/lib/python3.7/site-packages/opacus/privacy_engine.py:111: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


std:0.0030
mean:-0.00
std:0.0038
mean:0.00
std:0.0030
mean:0.00
std:0.0027
mean:-0.00
std:0.0030
mean:-0.00
std:0.0031
mean:0.00
std:0.0031
mean:0.00
std:0.0036
mean:-0.00
#####
std:0.0031
mean:-0.00
std:0.0038
mean:-0.00
std:0.0031
mean:-0.00
std:0.0031
mean:0.00
std:0.0031
mean:-0.00
std:0.0038
mean:-0.00
std:0.0033
mean:-0.00
std:0.0066
mean:0.00


In [127]:
d = list(train_dataloader)

In [131]:
id(model_opacus)

140455798254480

In [132]:
id(model_opacus_1)

140455795436752

In [163]:
import copy
import torchcsprng as csprng

random_number_generator = csprng.create_random_device_generator("/dev/urandom")

model_mine = Net()
model_mydp = copy.deepcopy(model_mine)
optimizer =  optim.SGD(model_mydp.parameters(), lr=args.lr)

model_nondp = copy.deepcopy(model_mine)
opt_nondp =  optim.SGD(model_nondp.parameters(), lr=args.lr)


data = train_dataloader.__iter__().next()[0]
target = train_dataloader.__iter__().next()[1]
output = model_mydp(data)
loss = F.cross_entropy(output, target)
loss.backward()
torch.nn.utils.clip_grad_norm_(model_mydp.parameters(),1.5)

for param in model_mydp.parameters():
    noise = torch.normal(0,20*1.5,param.size(),device=device, generator=random_number_generator)/args.batch_size*0.1
    param.grad.add_(noise)

optimizer.step()


data = train_dataloader.__iter__().next()[0]
target = train_dataloader.__iter__().next()[1]
output = model_nondp(data)
loss = F.cross_entropy(output, target)
loss.backward()
torch.nn.utils.clip_grad_norm_(model_nondp.parameters(),1.5)
opt_nondp.step()


compare_model(model_mine, model_mydp)
print('###########')
compare_model(model_mydp, model_nondp)

std:0.0032
mean:-0.00
std:0.0030
mean:-0.00
std:0.0030
mean:-0.00
std:0.0030
mean:-0.00
std:0.0030
mean:-0.00
std:0.0036
mean:-0.00
std:0.0029
mean:0.00
std:0.0085
mean:0.00
###########
std:0.0031
mean:0.00
std:0.0026
mean:0.00
std:0.0030
mean:0.00
std:0.0033
mean:-0.00
std:0.0030
mean:0.00
std:0.0033
mean:0.00
std:0.0028
mean:-0.00
std:0.0040
mean:-0.00


In [161]:
import copy
import torchcsprng as csprng

random_number_generator = csprng.create_random_device_generator("/dev/urandom")

model_mine = Net()
model_mydp = copy.deepcopy(model_mine)
optimizer =  optim.SGD(model_mydp.parameters(), lr=args.lr)

data = train_dataloader.__iter__().next()[0]
target = train_dataloader.__iter__().next()[1]
output = model_mydp(data)
loss = F.cross_entropy(output, target)
loss.backward()
torch.nn.utils.clip_grad_norm_(model_mydp.parameters(),1.5)

with torch.no_grad():
    for param in model_mydp.parameters():
        param.add_(torch.normal(0,6*1.5,param.size(),device=device)*args.lr/args.batch_size)
#                 param.add_(torch.from_numpy(np.random.normal(0,1.3*1.5,param.size())*args.lr).to(device))
optimizer.step()


compare_model(model_mine, model_mydp)

std:0.0012
mean:0.00
std:0.0008
mean:-0.00
std:0.0011
mean:-0.00
std:0.0013
mean:-0.00
std:0.0012
mean:0.00
std:0.0015
mean:0.00
std:0.0013
mean:0.00
std:0.0054
mean:0.00


In [51]:
train_dataloader.__iter__().next()[1]

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5,
        9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5, 6, 1, 0, 0, 1, 7,
        1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9, 0, 4, 6, 7, 4, 6, 8, 0,
        7, 8, 3, 1, 5, 7, 1, 7, 1, 1, 6, 3, 0, 2, 9, 3, 1, 1, 0, 4, 9, 2, 0, 0,
        2, 0, 2, 7, 1, 8, 6, 4, 1, 6, 3, 4, 5, 9, 1, 3, 3, 8, 5, 4, 7, 7, 4, 2,
        8, 5, 8, 6, 7, 3, 4, 6, 1, 9, 9, 6, 0, 3, 7, 2, 8, 2, 9, 4, 4, 6, 4, 9,
        7, 0, 9, 2, 9, 5, 1, 5, 9, 1, 2, 3, 2, 3, 5, 9, 1, 7, 6, 2, 8, 2, 2, 5,
        0, 7, 4, 9, 7, 8, 3, 2, 1, 1, 8, 3, 6, 1, 0, 3, 1, 0, 0, 1, 7, 2, 7, 3,
        0, 4, 6, 5, 2, 6, 4, 7, 1, 8, 9, 9, 3, 0, 7, 1, 0, 2, 0, 3, 5, 4, 6, 5,
        8, 6, 3, 7, 5, 8, 0, 9, 1, 0])

In [41]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296824


KeyboardInterrupt: 